# Kairos Demo Notebook

This notebook demonstrates the core features of Kairos, the AI/ML Cost Intelligence Layer.

## Installation

```bash
pip install -e kairos_plugin/
```

In [ ]:
# Import Kairos
from kairos import KairosTracker
import time

## 1. Initialize Tracker

Create a tracker with a custom hourly rate (or use cloud-specific configs).

In [ ]:
# Initialize with custom rate for demo
tracker = KairosTracker.with_rate(5.00)  # $5/hour

# Or use cloud-specific configuration:
# tracker = KairosTracker.for_aws("p4d.24xlarge")  # $32.77/hour
# tracker = KairosTracker.for_gcp("a2-highgpu-8g")  # $29.39/hour

print(f"Tracker initialized!")
print(f"Session ID: {tracker.session_id}")
print(f"Hourly Rate: ${tracker.hourly_rate:.2f}")
print(f"GPU Available: {tracker.gpu_available}")

## 2. View Current Status

Use `tracker.status()` to see current cost, runtime, and GPU utilization.

In [ ]:
# View current status - shows rich HTML display
tracker.status()

## 3. Log Cell Executions

### Method 1: Manual Logging with `log_cell()`

In [ ]:
# Simulate some work
print("Simulating data preprocessing...")
time.sleep(2)  # Simulate 2 seconds of work

# Log the cell execution
tracker.log_cell(
    duration=2.0,
    cell_number=1,
    tags=["preprocessing", "data"]
)

In [ ]:
# Simulate training
print("Simulating model training...")
time.sleep(3)  # Simulate 3 seconds of training

tracker.log_cell(
    duration=3.0,
    cell_number=2,
    tags=["training", "model"]
)

### Method 2: Automatic Tracking with `track_cell()` Context Manager

In [ ]:
# Use context manager for automatic timing
with tracker.track_cell(cell_number=3, tags=["inference"]) as cell:
    print("Simulating inference...")
    time.sleep(1.5)
    print("Inference complete!")

print(f"\nCell cost: ${cell.cost_usd:.4f}")
print(f"Duration: {cell.duration_seconds:.2f}s")

In [ ]:
# Another tracked cell
with tracker.track_cell(cell_number=4, tags=["evaluation", "metrics"]):
    print("Computing metrics...")
    time.sleep(2)
    print("Metrics computed!")

## 4. Check Status Again

See updated costs after running cells.

In [ ]:
# Updated status
tracker.status()

## 5. View Session Summary

Get a comprehensive summary with cost breakdown, efficiency metrics, and recommendations.

In [ ]:
# View comprehensive summary
summary = tracker.summary()

print(f"\n--- Summary Data ---")
print(f"Total Cost: ${summary.total_cost_usd:.4f}")
print(f"Total Runtime: {summary.total_duration_seconds:.1f}s")
print(f"Cells Executed: {summary.cell_count}")
print(f"Efficiency Score: {summary.efficiency_score:.0f}/100")

## 6. Get Cost Projections

In [ ]:
# Get cost projections
proj = tracker.get_projection(hours=1)

print("Cost Projections:")
print(f"  Current:  ${proj['current_cost_usd']:.4f}")
print(f"  +1 hour:  ${proj['projected_1h_usd']:.2f}")
print(f"  +8 hours: ${proj['projected_8h_usd']:.2f}")
print(f"  +24 hours: ${proj['projected_24h_usd']:.2f}")
print(f"\nMonthly rate at current usage: ${proj['monthly_rate_usd']:.2f}")

## 7. Export Data

In [ ]:
# Export all data as dictionary
data = tracker.to_dict()

print("Exported data keys:", list(data.keys()))
print(f"\nNumber of cells logged: {len(data['cells'])}")

# Show cell costs
print("\nCell Costs:")
for cell in data['cells']:
    print(f"  Cell #{cell['cell_number']}: ${cell['cost_usd']:.4f} ({cell['duration_seconds']:.1f}s) - {cell['tags']}")

## 8. Budget Alerts

In [ ]:
# Set alert threshold
tracker.set_alert_threshold(0.01)  # Alert at $0.01

# Check budget
exceeded = tracker.check_budget(0.001)  # Very low budget to trigger
print(f"\nBudget exceeded: {exceeded}")

## 9. Custom Alerts

In [ ]:
# Display custom alerts
tracker.alert("This is an informational message", alert_type="info")
tracker.alert("Warning: GPU utilization is low", alert_type="warning")
tracker.alert("Error: Budget exceeded!", alert_type="error")

## 10. End Session

In [ ]:
# End the session and get final summary
final_summary = tracker.end_session()

print("Session ended!")
print(f"Final cost: ${final_summary.total_cost_usd:.4f}")
print(f"Total runtime: {final_summary.total_duration_seconds:.1f}s")

---

## Cloud Provider Examples

Here are examples for different cloud providers:

In [ ]:
# AWS Example
aws_tracker = KairosTracker.for_aws("p4d.24xlarge")
print(f"AWS p4d.24xlarge: ${aws_tracker.hourly_rate}/hour")

# GCP Example  
gcp_tracker = KairosTracker.for_gcp("a2-highgpu-8g")
print(f"GCP a2-highgpu-8g: ${gcp_tracker.hourly_rate}/hour")

# Azure Example
azure_tracker = KairosTracker.for_azure("NC24ads_A100_v4")
print(f"Azure NC24ads_A100_v4: ${azure_tracker.hourly_rate}/hour")

---

## That's It!

You've learned how to:
- Initialize a Kairos tracker
- View real-time status
- Log cell executions manually or automatically
- Get session summaries with recommendations
- Set up cost alerts
- Export data for analysis

For more information, visit [usekairos.ai](https://usekairos.ai)